In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from torchfm.model import TorchFM
from torchfm.model import ModelTrainer
from torchfm.data import movielens100k

In [3]:
train = movielens100k.load(train=True)
test = movielens100k.load(train=False)

In [4]:
model = TorchFM(
    train.user_input_dim,
    train.item_input_dim
)
trainer = ModelTrainer(
    model,
    dataset=train,
    learning_rate=1e-3
)
trainer.train()

2024-11-30 19:21:13.530 | INFO     | torchfm.model._trainer:train:38 - Completed create DataLoader.
Training: 100%|██████████| 5/5 [00:10<00:00,  2.08s/it, batch=625, loss=0.8905]
2024-11-30 19:21:23.930 | INFO     | torchfm.model._trainer:train:57 - Total training time: 10.40 seconds


In [15]:
import numpy as np
from torchfm.model.evaluation import auc_score
auc_scores = []
for user_id in train.get_all_user_ids():
    item_ids = train.get_all_item_ids()
    scores = []
    for item_id in item_ids:
        score = trainer.predict_by_id(user_id, item_id)
        scores.append(score)

    ground_truth = np.zeros((len(item_ids), ))
    for x, _ in train.get_interacted_items(user_id):
        for i, t in enumerate(item_ids):
            if x == t:
                ground_truth[i] = 1

    auc_scores.append(auc_score(np.array(scores), ground_truth))